In [ ]:
__author__ = "Alireza Sadabadi"
__copyright__ = "Copyright (c) 2024 Alireza Sadabadi. All rights reserved."
__credits__ = ["Alireza Sadabadi"]
__license__ = "Apache"
__version__ = "2.0"
__maintainer__ = "Alireza Sadabadi"
__email__ = "alirezasadabady@gmail.com"
__status__ = "Test"
__doc__ = "you can see the tutorials in https://youtube.com/@alirezasadabadi?si=d8o7LK_Ai1Hf68is"

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import time
from Meta import *
from colorama import init as colorama_init
from colorama import Fore
from colorama import Style
import pandas_ta as pdta
import socket
import sys
from scipy.stats import linregress
import joblib

In [2]:
colorama_init()
# ساخت کانکشن بین ربات و متاتریدر
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

def internet(host="8.8.8.8", port=53, timeout=3):
    """
    Host: 8.8.8.8 (google-public-dns-a.google.com)
    OpenPort: 53/tcp
    Service: domain (DNS/TCP)
    """
    try:
        socket.setdefaulttimeout(timeout)
        socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
        return True
    except socket.error as ex:
        # print(f"Internet Connection Error: {ex}")
        # print("Don't worried, I will try 10 seconds later again :)")
        print("@",end='')
        sys.stdout.flush()
        return False
def GetSlope(array):
    y = np.array(array)
    x = np.arange(len(y))
    response = linregress(x,y)
    return response.slope

In [3]:
def Strategy(symbol, preBuy, preSell, status):
    number_of_data = 170
    try:
        df = Meta.GetRates(symbol, number_of_data , timeFrame=mt5.TIMEFRAME_H4)
    except BaseException as e:
        print(f"An exception has occurred in TraderBot.BB_Full GetRates: {str(e)}")
        return preBuy, preSell, status
    else:
        # فرمول استراتژی برای سیگنال و تولید ستون های لازم
        df['ATR'] = pdta.atr(df['high'],df['low'],df['close'],length=20)
        df['RSI'] = pdta.rsi(df['close'])
        df['MA40'] = pdta.sma(df['close'],length=40)
        df['MA80'] = pdta.sma(df['close'],length=80)
        df['MA160'] = pdta.sma(df['close'],length=160)
        df['Average'] = pdta.midprice(df['high'],df['low'])
        
        backRolling = 7    
        df['ATRSlope'] = df['ATR'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df['RSISlope'] = df['RSI'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df['MA40Slope'] = df['MA40'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df['MA80Slope'] = df['MA80'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df['MA160Slope'] = df['MA160'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df['AverageSlope'] = df['Average'].rolling(window=backRolling).apply(GetSlope, raw=True)
        df.dropna(inplace=True)
        #اگر پوزیشن خرید و فروش باز از قبل نداریم
        if status == False:
            model = joblib.load('Model-XGBOOST.ml')
            attributes=['ATR', 'RSI', 'Average', 'MA40', 'MA80', 'MA160', 'MA40Slope', 'MA80Slope', 'MA160Slope', 'AverageSlope', 'RSISlope']
            prediction = model.predict(df[attributes])
            print(prediction)
            # تشکیل سیگنال خرید و فروش
            if prediction[-2] == 2:
                buy = True
                sell = False
            if prediction[-2] == 1:
                sell = True
                buy = False
            if prediction[-2] == 0:
                buy = False
                sell = False
            
            if buy == True or sell == True:
                status = True
            return buy,sell,status
        else:
            return preBuy, preSell, status

In [ ]:
accountInfo = mt5.account_info()
print("-"*75)
print(f"Login: {accountInfo.login} \tserver: {accountInfo.server} \tleverage: {accountInfo.leverage}")
print(f"Balance: {accountInfo.balance} \tEquity: {accountInfo.equity} \tProfit: {accountInfo.profit}")
print(f"Run time: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')}")
print("-"*75)

symbols_list = {
    "EURUSD": ["EURUSD", 0.01],
   }

buy = False
sell = False
status = False
magic = 7

while True:

    if internet() == True:
        
        Meta.TrailingStopLoss([magic])
        Meta.VerifyTSL([magic])

         # Strategy
        for asset in symbols_list.keys():
            symbol = symbols_list[asset][0]
            lot = symbols_list[asset][1]

            selected = mt5.symbol_select(symbol)
            if not selected:
                print(f"\nERROR - Failed to select '{symbol}' in MetaTrader 5 with error :",mt5.last_error())                
            else:         
                resume = Meta.resume()
                if resume.shape[0] > 0:
                    row = resume.loc[(resume["symbol"] == symbol) & (resume["magic"] == magic)]
                    # در صورتی که استاپ لاس یک پوزیشن بخوره
                    # باید وضعیت به حالت اولیه برای سفارش گذاری برگرده
                    if row.empty and status == True:
                        status=False
                        print(f"Strategy {Fore.YELLOW}StopLoss hit!{Style.RESET_ALL}")
                        # حلقه اصلی هر ۱۰ ثانیه اجرا می شود بنابراین اگر در 
                        # موقعیتی استاپ لاس خورد دوباره در همان موقعیت نباید
                        # پوزیشن قبلی مجدد باز شود
                        time.sleep(290)
                    elif not row.empty and status == False:
                        print("Abnormally position: you have a open position with Strategy strategy but the status key is False!!")
                elif status == True:
                    status=False
                    print(f"Strategy {Fore.YELLOW}StopLoss hit!{Style.RESET_ALL}")
                    time.sleep(290)

                buy,sell,status=Strategy(symbol, buy, sell, status)
                Meta.run(symbol, buy, sell, lot, 100*1e-4, 50*1e-4, magic)  

    # سیگنال زنده بودن ربات                
    # counter += 1          
    # print(f"{':' if counter % 2 == 0 else '.'}",end='')
    # sys.stdout.flush()

    
    time.sleep(10)

---------------------------------------------------------------------------
Login: 52359843 	server: Alpari-MT5-Demo 	leverage: 100
Balance: 99999.92 	Equity: 99999.92 	Profit: 0.0
Run time: 2025-04-30 17:23:57
---------------------------------------------------------------------------
[2 2 2 2 2]
askPrice:1.13625
---------------------------------------------------------------------------
Date:  2025-04-30 17:23:58 	SYMBOL: EURUSD
BUY: True 	  SELL: False  	 Magic: 7
OPEN BUY POSITION: Request executed
price: 1.13625 	 SL: 1.13125 	 TP: 1.14625
---------------------------------------------------------------------------
Buy StopLoss Trailing	symbol:EURUSD	magic:7	order:259748697	price:1.13625	SL:1.131265
Buy StopLoss Trailing	symbol:EURUSD	magic:7	order:259748697	price:1.13625	SL:1.13135
Buy StopLoss Trailing	symbol:EURUSD	magic:7	order:259748697	price:1.13625	SL:1.1314850000000003
Buy StopLoss Trailing	symbol:EURUSD	magic:7	order:259748697	price:1.13625	SL:1.1314949999999997
@@askPrice